In [1]:

import torch
import requests
from PIL import Image
from transformers import Blip2Processor, Blip2ForConditionalGeneration

# processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
# model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16, device_map="auto")

processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xl", torch_dtype=torch.float16, device_map="auto")

C:\Users\andy5\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:30<00:00, 15.42s/it]
C:\Users\andy5\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\andy5\.cache\huggingface\hub\models--Salesforce--blip2-flan-t5-xl. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` env

In [ ]:
raw_image = Image.open("C:/Users/andy5/Desktop/cvpdl_hw3/images/construction-site-build-construction-work-159306.jpeg").convert('RGB')
# raw_image.show()
question = "Please describe the content of this image in detail"
inputs = processor(raw_image, question, return_tensors="pt").to("cuda")

out = model.generate(**inputs,max_length=400,min_length=50)
print(processor.decode(out[0], skip_special_tokens=True).strip())

question = "Please describe the content of this image in detail"#two construction workers working on a construction site
inputs = processor(raw_image, question, return_tensors="pt").to("cuda")

out = model.generate(**inputs,max_length=200)
print(processor.decode(out[0], skip_special_tokens=True).strip())


two construction workers working on a construction site in the city of london, uk - stock photo -
two construction workers working on a construction site


In [8]:
import json
import os
from PIL import Image
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
def load_labels(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    return data


# 使用BLIP2模型進行圖像描述生成
def generate_caption(model, processor, image_path):
    image = Image.open(image_path).convert('RGB')
    inputs = processor(images=image, return_tensors="pt").to("cuda", torch.float16)
    output = model.generate(**inputs)
    caption = processor.decode(output[0], skip_special_tokens=True)
    return caption

# 保存結果為jsonl檔案
def save_to_jsonl(data, output_path):
    with open(output_path, 'w') as f:
        for entry in data:
            f.write(json.dumps(entry) + '\n')

In [10]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
json_path = 'label.json'  # 請替換為你的label.json路徑
images_dir = 'images'  # 圖片的目錄，請確保包含label.json中的圖片
output_path = 'output_blip2-flan-t5-xl.jsonl'

In [12]:
# 讀取label.json資料
labels_data = load_labels(json_path)
import time
from PIL import Image
from IPython.display import display
# 遍歷每張圖片，生成描述並保存結果
output_data = []
for item in labels_data:
    image_name = item['image']
    image_path = os.path.join(images_dir, image_name)

    if os.path.exists(image_path):
        raw_image = Image.open(image_path).convert('RGB')
        question = "Please describe the image in detail"
        inputs = processor(raw_image, question, return_tensors="pt").to("cuda", torch.float16)
        out = model.generate(**inputs,max_length=200)
        caption=processor.decode(out[0], skip_special_tokens=True)
        print("生成描述:",caption)
        print(processor.decode(out[0], skip_special_tokens=True))
        # caption = generate_caption(model, processor, image_path)
        item['generated_text'] = caption
        print(f"生成描述: {caption} (圖片: {image_name})")
    else:
        print(f"圖片不存在: {image_name}")
    output_data.append(item)
inputs = processor(raw_image, question, return_tensors="pt").to("cuda", torch.float16)

# 保存為jsonl檔案
save_to_jsonl(output_data, output_path)
print(f"生成的描述已保存至 {output_path}")

生成描述: two construction workers working on a construction site
two construction workers working on a construction site
生成描述: two construction workers working on a construction site (圖片: construction-site-build-construction-work-159306.jpeg)
生成描述: a worker is working on a wooden plank
a worker is working on a wooden plank
生成描述: a worker is working on a wooden plank (圖片: construction-site-build-construction-work-159375.jpeg)
生成描述: a man walking with a large blower
a man walking with a large blower
生成描述: a man walking with a large blower (圖片: gardener-worker-gardening-machinery-162564.jpeg)
生成描述: a blue tractor is driving a large blue combine harvester
a blue tractor is driving a large blue combine harvester
生成描述: a blue tractor is driving a large blue combine harvester (圖片: harvest-grain-combine-arable-farming-163752.jpeg)
生成描述: a man is working on a machine
a man is working on a machine
生成描述: a man is working on a machine (圖片: pexels-photo-10039988.jpeg)
生成描述: a woman is working in a fac